# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. 

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
# количество пропущенных значений для каждого столбца
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [6]:
# заполняем пропуски в стобце total_income
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
# уберем отрицательные значения количества дней трудового стажа 
data['days_employed'] = data['days_employed'].abs()

In [8]:
# для каждого типа занятости выведем медианное значение трудового стажа
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
# выведем уникальные значения столбца children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
# удаляем строки с аномальными значениями
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)¶

In [12]:
# заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
# заменим вещественный тип данных в столбце total_income на целочисленный 
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру и удалим дубликаты.

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Расспределим все цели кредита по 4 основным категориям:**
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

In [20]:
# перечень уникальных целей взятия кредита из столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
data['children'].value_counts()  # сгруппируем заемщиков по количеству детей и посмотрим на сколько объемен каждый класс

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [24]:
data.groupby('children')['debt'].sum()  # посмотрим на количество должников в каждом классе

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [25]:
data.pivot_table(index='children', values='debt', aggfunc='mean')  # рассчитаем долю должников в каждом классе от общего количества заемщиков

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


In [26]:
data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])  # составим сводную таблицу по всем значениям

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** 

Поскольку количество заемщиков в каждом классе значительно отличается, оценивать зависимость между количеством детей и возвратом кредита в срок только по количеству должников в каждом классе было бы ошибочным.

Рассмотрим общее количество заемщиков и дюлю должнеков в каждом классе. Мы можем сделать следующие выводы:
* Больше всего берут кредиты люди, у которых нет детей. Это вполне ожидаемо, так как наличие детей вызывает дополнительные расходы, из-за которых людям труднее решиться на займ. Также в данном классе невысокая доля должников.
* Многодетные люди (3 и более детей) режи берут кредиты, чем немногодетные.
* Количество должников среди немногодетных заемщиков (до 3 детей) растет с увеличением количества детей.
* Доля должников следи заемщиков с 5 детьми равна 0. Однако общее количество людей в этом классе также невелико - всего 9. Следовательно в данном случае можно предполижить, что люди с таким количеством детей очень здраво оценивают свои возможности и берут кредит, только если точно уверены в своей способности его выплатить.
* Самый большой процент должников среди людей с 4 детьми. Однако данная категория также немногочисленна в сравнение с другими.

Таким образом, можно сказать, что зависимость между количеством детей и возвратом кредита в срок присутствует.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data['family_status'].value_counts()  # сгруппируем заемщиков по их семейному положению и посмотрим на сколько объемен каждый класс

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [28]:
data.groupby('family_status')['debt'].sum()  # посмотрим на количество должников в каждом классе

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

In [29]:
data.pivot_table(index='family_status', values='debt', aggfunc='mean')  # рассчитаем долю должников в каждом классе от общего количества заемщиков

,debt
family_status,
Не женат / не замужем,0.097639
в разводе,0.070648
вдовец / вдова,0.066246
гражданский брак,0.093130
женат / замужем,0.075606


In [30]:
data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean'])  # составим сводную таблицу по всем значениям

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** 

Рассмотрим общее количество заемщиков и дюлю должников в каждом классе. Мы можем сделать следующие выводы:
* Чаще всего берут кредиты люди находящиеся в браке. Это закономерно, так как имущество приобретенное на кредитные средства будет совместно нажитым, и выплачивать такой долг каждый из супругов будет не в одиночку. 
* Одинокие реже бедут кредиты, чем люди имеющие пару.
* Самая маленькая доля должников среди вдовцов/вдов, но также это самая малочисленная группа среди заемщиков.
* Самая высокая доля должников среди несостоявших в браке (не женат/не замужем).

Какой-либо четкой зависимости между семейным положением и возвратом кредита в срок нет, `но можно сказать, что люди, состоящие или состоявшие в браке, более надежные заемщики`.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?¶

In [31]:
data['total_income_category'].value_counts()  # группируем заемщиков по категориям уровня дохода и посмотрим на сколько объемен каждый класс

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

In [32]:
data.groupby('total_income_category')['debt'].sum()  # посмотрим на количество должников в каждом классе

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64

In [33]:
data.pivot_table(index='total_income_category', values='debt', aggfunc='mean')  # рассчитаем долю должников в каждом классе от общего количества заемщиков

,debt
total_income_category,
A,0.080000
B,0.070602
C,0.084982
D,0.060172
E,0.090909


In [34]:
data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])  # составим сводную таблицу по всем значениям 

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** 

Рассмотрим общее количество заемщиков и дюлю должников в каждом классе. Мы можем сделать следующие выводы:
* Чаще всего берут кредиты люди из средней группы по доходам (C). Это ожидаемо, так как средний класс не имеет достаточно средств для любых своих нужд, но при этом имеет финансовую возможность выплатить долг. Однако в этом классе также достаточно большая доля должников.
* `Еще одна многочисленная группа - это группа с доходом ниже средней(B). Кроме того доля должников в ней заметно меньше, чем в среднем классе. Это может быть связано с тем, что данный класс уже имеет достаточный для выплаты уровень дохода, но при этом недостаточно большой, чтобы переоценить свои возможности.`
* Реже всего берут кредиты люди с самым маленьким и самым большим доходом. Эти же группы имеют достаточно большую долю должников.

Четкая зависимость между уровнем дохода и возвратом кредита в срок отсутствует.

### Как разные цели кредита влияют на его возврат в срок?

In [35]:
data['purpose_category'].value_counts()  # группируем заемщиков по цели кредита и посмотрим на сколько объемен каждый класс

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

In [36]:
data.groupby('purpose_category')['debt'].sum()  # посмотрим на количество должников в каждом классе

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [37]:
data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')  # рассчитаем долю должников в каждом классе от общего количества заемщиков

,debt
purpose_category,
операции с автомобилем,0.093480
операции с недвижимостью,0.072551
получение образования,0.092528
проведение свадьбы,0.079118


In [38]:
data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])  # составим сводную таблицу по всем значениям 

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** 

* Чаще всего кредит берут на `операции с недвижимостью`, при этом в данном классе самая малая доля должников. Это можно попробовать объяснить тем, что кредит на недвижимость берут люди с достаточно стабильным доходом на долгий срок. Кроме того жилая недвижимость - это жизненно необходимое имущество.
* Реже всего кредит берут на `проведение свадьбы`. Это скорее всего связанно с тем, что свадьбы - это не имущество, а событие, которое люди часто стараются организовать в пределах доступных средств. Доля должников в данном классе также достаточно мала, что можно объястнить тем, что суммы кредитов в данной категории скорее всего небольшие.
* Самые большые доли должников в категориях `'операции с автомобилем'` и `'получение образования'`.

### Возможные причины появления пропусков в исходных данных.

Пропуски бывают 3 типов:

* **Полностью случайные**: вероятность появления такого пропуска не зависит не от каких других значений. Такие пропуски могут появиться, например, из-за какой-то технической ошибки при копировании, считывании или смене формата данных. Также здесь может быть задействован человеческий фактор. Например, медицинская карта какого-либо пациента может быть не внесяна в базу потому, что какой-то сотрудник ее просто потерял.
* **Случайные**: вероятность появления такого пропуска зависит от других значений в наборе данных, но не от самого столбца с пропуском. Например, данные о доходах человека могут отсутствовать, если этот человек - ребенок.
* **Неслучайные**: вероятность появления такого пропуска зависит от других значений, в том числе и от значений собственного столбца. Например, в таблице с источниками трафика сайта может отсутствовать email пользователя, потому что он сам не захотел подписываться на рассылку, а на сайт попадает например из поисковой выдачи.

**В рассматриваемых данных есть пропуски в 2 типах значений:** общий трудовой стаж в днях и ежемесячный доход. Оба этих значения могут отвутствовать по причине случайного невнесения данных или из-за какой-то технической ошибки. Кроме того трудовой стаж может отсутствовать, если трудоустройство было неофициальным и не занесено в трудовую книжку.

### Причина почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Медиана** - это число, делящее упорядоченную по возрастанию выборку попалам, то есть половина значений в выборке меньше медианы, а другая половина - больше.

Для заполнения пропусков в количественных переменных медиана подходит лучше, чем среднее арифметическое, потому что среднее некорректно характеризует данные, когда в них есть сильно выделяющиеся значения. Например, нам нужно узнать средний возраст в следующей группе людей `[20, 20, 20, 20, 20, 64]`. Если мы возьмем среднее арифметическое, то он будет равен `27`, однако медиана будет равна `20`, что гораздо лучше отражает действительность. 

## Общий вывод

Исследование данных на предмет зависимости возврата кредита в срок от различных критериев, характеризующих заемщиков, показало, что более менее явная взаимосвязь присутствует в случае с количеством детей: чем больше детей, тем реже люди способны выплатить кредит. Однако во всех остальных случаях слишком высок человеческий фактор и различные дополнительные жизненные обстоятельства. То есть, например, нельзя одназначно утверждать, что если человек вдовец, то он точно вернет кредит.

В итоге, можно сказать, что **под "идеальный" портрет клиента попадают люди, состоящие или состоявшие в браке, при этом не имеющие детей**. Их доходы должен быть выше или ниже среднего (категории B и D). Самые же "безопасные" для банка цели кредита - это недвижимость и свадьба.

**Самые ненадежные клиенты - это многодетные или ненаходящиеся в браке заемщики, а самые "опасные" цели кредита - автомобиль и образование**.